# Spark

Copyright Felix Martin Schuhknecht, Jens Dittrich & Marcel Maltry  [Big Data Analytics Group](https://bigdata.uni-saarland.de/), [CC-BY-SA](https://creativecommons.org/licenses/by-sa/4.0/legalcode)

The following notebook was tested with Spark 2.4.3 and requires SPARK_HOME to be set to the Spark path

### Imports

In [1]:
import os
from graphviz import Digraph, Source
from ra.relation import Relation
from ra.schema_utils import build_schema
from os import listdir
from ra.operators_spark import *

### Create a new Spark session

In [2]:
session = SparkSession.builder.getOrCreate()

### Application

In [3]:
# Data source: https://relational.fit.cvut.cz/dataset/IMDb
# Information courtesy of IMDb (http://www.imdb.com). Used with permission.
# Notice: The data can only be used for personal and non-commercial use and must not
# be altered/republished/resold/repurposed to create any kind of online/offline
# database of movie information (except for individual personal use).

path = 'data/IMDb_sample'  
# create a list of all files in that directory that end with "*.csv":
files = [file for file in listdir(path) if file.endswith('.csv')]

from ra.csv_utils import load_csv

relations = [load_csv(path + '/' + file, file[:-4], delimiter='\t') for file in files]

relationsDict = {}
for rel in relations:
    relationsDict[rel.name] = rel
print(relationsDict)

{'movies_directors': <ra.relation.Relation object at 0x11dc37fd0>, 'actors': <ra.relation.Relation object at 0x11dc33160>, 'directors': <ra.relation.Relation object at 0x11dcb72e8>, 'movies_genres': <ra.relation.Relation object at 0x11dcb7438>, 'directors_genres': <ra.relation.Relation object at 0x11dcbc6a0>, 'movies': <ra.relation.Relation object at 0x11dcbcdd8>, 'roles': <ra.relation.Relation object at 0x11dcbcd30>}


We build LeafSpark objects from existing relations. Each LeafSpark object contains the data in form of a Spark DataFrame. 

In [4]:
# build LeafRelation object from relations dictionary
movies_directors = LeafSpark(relationsDict['movies_directors'], session)
actors = LeafSpark(relationsDict['actors'], session)
directors = LeafSpark(relationsDict['directors'], session)
movies_genres = LeafSpark(relationsDict['movies_genres'], session)
directors_genres = LeafSpark(relationsDict['directors_genres'], session)
movies = LeafSpark(relationsDict['movies'], session)
roles = LeafSpark(relationsDict['roles'], session)

In [5]:
directors.evaluate().show()
directors.evaluate().printSchema()

+-----+----------+---------+
|   id|first_name|last_name|
+-----+----------+---------+
|11652| James (I)|  Cameron|
|78273|   Quentin|Tarantino|
|43095|   Stanley|  Kubrick|
+-----+----------+---------+

root
 |-- id: integer (nullable = false)
 |-- first_name: string (nullable = false)
 |-- last_name: string (nullable = false)



# Selection

In [6]:
newmovies = Selection_Spark(movies, 'year>2000')

In [7]:
newmovies.evaluate().show()

+------+--------------------+----+----+
|    id|                name|year|rank|
+------+--------------------+----+----+
| 10934|  Aliens of the Deep|2005| 6.5|
|105938|Expedition: Bismarck|2002| 7.5|
|159665| Inglorious Bastards|2006| 8.3|
| 96779|        Earthship.TV|2001| 5.6|
|393538|  Jimmy Kimmel Live!|2003| 6.7|
|127297| Ghosts of the Abyss|2003| 6.7|
|176711|   Kill Bill: Vol. 1|2003| 8.4|
|176712|   Kill Bill: Vol. 2|2004| 8.2|
+------+--------------------+----+----+



# Projection

In [8]:
exp2 = Projection_Spark(newmovies, ['id', 'year'])

In [9]:
exp2.evaluate().show()

+------+----+
|    id|year|
+------+----+
| 10934|2005|
|105938|2002|
|159665|2006|
| 96779|2001|
|393538|2003|
|127297|2003|
|176711|2003|
|176712|2004|
+------+----+



# Cartesian Product

In [10]:
cartesianProduct = Cartesian_Product_Spark(directors, directors_genres)

In [11]:
directors.evaluate().show()

+-----+----------+---------+
|   id|first_name|last_name|
+-----+----------+---------+
|11652| James (I)|  Cameron|
|78273|   Quentin|Tarantino|
|43095|   Stanley|  Kubrick|
+-----+----------+---------+



In [12]:
directors_genres.evaluate().show()

+-----------+-----------+---------+
|director_id|      genre|     prob|
+-----------+-----------+---------+
|      11652|     Horror| 0.166667|
|      78273|     Action|      0.5|
|      11652|   Thriller| 0.416667|
|      11652|Documentary|     0.25|
|      78273|      Crime|      0.5|
|      43095|     Action|   0.0625|
|      11652|      Drama|     0.25|
|      78273|    Romance|    0.125|
|      43095|      Music|   0.0625|
|      43095|    Mystery|   0.0625|
|      78273|     Comedy|     0.25|
|      78273|      Drama|     0.75|
|      43095|     Horror|   0.0625|
|      43095|        War|    0.375|
|      78273|   Thriller|      0.5|
|      11652|     Sci-Fi|      0.5|
|      43095|      Crime|   0.1875|
|      78273|        War|    0.125|
|      11652|    Fantasy|0.0833333|
|      43095|     Sci-Fi|   0.1875|
+-----------+-----------+---------+
only showing top 20 rows



In [13]:
cartesianProduct.evaluate().show(10)

+-----+----------+---------+-----------+-----------+--------+
|   id|first_name|last_name|director_id|      genre|    prob|
+-----+----------+---------+-----------+-----------+--------+
|11652| James (I)|  Cameron|      11652|     Horror|0.166667|
|11652| James (I)|  Cameron|      78273|     Action|     0.5|
|11652| James (I)|  Cameron|      11652|   Thriller|0.416667|
|11652| James (I)|  Cameron|      11652|Documentary|    0.25|
|11652| James (I)|  Cameron|      78273|      Crime|     0.5|
|11652| James (I)|  Cameron|      43095|     Action|  0.0625|
|11652| James (I)|  Cameron|      11652|      Drama|    0.25|
|11652| James (I)|  Cameron|      78273|    Romance|   0.125|
|11652| James (I)|  Cameron|      43095|      Music|  0.0625|
|11652| James (I)|  Cameron|      43095|    Mystery|  0.0625|
+-----+----------+---------+-----------+-----------+--------+
only showing top 10 rows



# Intersection

In [14]:
goodmovies = Selection_Spark(movies, "rank>=7.5")
goodAndNew = Intersection_Spark(newmovies, goodmovies)

In [15]:
goodAndNew.evaluate().show()

+------+--------------------+----+----+
|    id|                name|year|rank|
+------+--------------------+----+----+
|176711|   Kill Bill: Vol. 1|2003| 8.4|
|105938|Expedition: Bismarck|2002| 7.5|
|159665| Inglorious Bastards|2006| 8.3|
|176712|   Kill Bill: Vol. 2|2004| 8.2|
+------+--------------------+----+----+



Alternatively, without intersection but with two conditions in the selection instead:

In [16]:
goodAndNewSel = Selection_Spark(movies, "year>2000 and rank>=7.5")

In [17]:
goodAndNewSel.evaluate().show()

+------+--------------------+----+----+
|    id|                name|year|rank|
+------+--------------------+----+----+
|105938|Expedition: Bismarck|2002| 7.5|
|159665| Inglorious Bastards|2006| 8.3|
|176711|   Kill Bill: Vol. 1|2003| 8.4|
|176712|   Kill Bill: Vol. 2|2004| 8.2|
+------+--------------------+----+----+



# Union

In [18]:
goodOrNew = Union_Spark(goodmovies, newmovies)

In [19]:
goodOrNew.evaluate().show()

+------+--------------------+----+----+
|    id|                name|year|rank|
+------+--------------------+----+----+
|310455|           Spartacus|1960| 8.0|
|121538|   Full Metal Jacket|1987| 8.2|
| 10920|              Aliens|1986| 8.2|
|164572|        Jackie Brown|1997| 7.5|
|  1711|2001: A Space Ody...|1968| 8.3|
|105938|Expedition: Bismarck|2002| 7.5|
|328285|     Terminator, The|1984| 7.9|
|159665| Inglorious Bastards|2006| 8.3|
|328277|Terminator 2: Jud...|1991| 8.1|
|387728|                  ER|1994| 7.7|
| 92616|Dr. Strangelove o...|1964| 8.7|
|250612|      Paths of Glory|1957| 8.6|
| 65764| Clockwork Orange, A|1971| 8.3|
| 30431|        Barry Lyndon|1975| 7.9|
|177019|        Killing, The|1956| 8.1|
|193519|              Lolita|1962| 7.6|
|176711|   Kill Bill: Vol. 1|2003| 8.4|
|176712|   Kill Bill: Vol. 2|2004| 8.2|
|267038|        Pulp Fiction|1994| 8.7|
|276217|      Reservoir Dogs|1992| 8.3|
+------+--------------------+----+----+
only showing top 20 rows



Alternatively, without union but with two conditions in the selection instead.

In [20]:
goodOrNewSel = Selection_Spark(movies, "year>2000 or rank>=7.5")

In [21]:
goodOrNewSel.evaluate().show()

+------+--------------------+----+----+
|    id|                name|year|rank|
+------+--------------------+----+----+
|310455|           Spartacus|1960| 8.0|
| 10934|  Aliens of the Deep|2005| 6.5|
|121538|   Full Metal Jacket|1987| 8.2|
| 10920|              Aliens|1986| 8.2|
|164572|        Jackie Brown|1997| 7.5|
|  1711|2001: A Space Ody...|1968| 8.3|
|105938|Expedition: Bismarck|2002| 7.5|
|328285|     Terminator, The|1984| 7.9|
|159665| Inglorious Bastards|2006| 8.3|
|328277|Terminator 2: Jud...|1991| 8.1|
|387728|                  ER|1994| 7.7|
| 92616|Dr. Strangelove o...|1964| 8.7|
|250612|      Paths of Glory|1957| 8.6|
| 65764| Clockwork Orange, A|1971| 8.3|
| 30431|        Barry Lyndon|1975| 7.9|
| 96779|        Earthship.TV|2001| 5.6|
|177019|        Killing, The|1956| 8.1|
|193519|              Lolita|1962| 7.6|
|393538|  Jimmy Kimmel Live!|2003| 6.7|
|127297| Ghosts of the Abyss|2003| 6.7|
+------+--------------------+----+----+
only showing top 20 rows



# Difference

In [22]:
newButBadMovies = Difference_Spark(newmovies, goodmovies)

In [23]:
newButBadMovies.evaluate().show()

+------+-------------------+----+----+
|    id|               name|year|rank|
+------+-------------------+----+----+
|393538| Jimmy Kimmel Live!|2003| 6.7|
|127297|Ghosts of the Abyss|2003| 6.7|
| 10934| Aliens of the Deep|2005| 6.5|
| 96779|       Earthship.TV|2001| 5.6|
+------+-------------------+----+----+



Alternatively, without intersection but with two conditions in the selection instead.

In [24]:
newButBadMoviesSel = Selection_Spark(movies, "year>2000 and not rank>=7.5")

In [25]:
newButBadMoviesSel.evaluate().show()

+------+-------------------+----+----+
|    id|               name|year|rank|
+------+-------------------+----+----+
| 10934| Aliens of the Deep|2005| 6.5|
| 96779|       Earthship.TV|2001| 5.6|
|393538| Jimmy Kimmel Live!|2003| 6.7|
|127297|Ghosts of the Abyss|2003| 6.7|
+------+-------------------+----+----+



# Renaming Relation

In [26]:
# DataFrames do not have a name. Thus, renaming has no effect
exp11 = Renaming_Relation_Spark(goodOrNew, "good_or_new")

In [27]:
exp11.evaluate().show()

+------+--------------------+----+----+
|    id|                name|year|rank|
+------+--------------------+----+----+
|310455|           Spartacus|1960| 8.0|
|121538|   Full Metal Jacket|1987| 8.2|
| 10920|              Aliens|1986| 8.2|
|164572|        Jackie Brown|1997| 7.5|
|  1711|2001: A Space Ody...|1968| 8.3|
|105938|Expedition: Bismarck|2002| 7.5|
|328285|     Terminator, The|1984| 7.9|
|159665| Inglorious Bastards|2006| 8.3|
|328277|Terminator 2: Jud...|1991| 8.1|
|387728|                  ER|1994| 7.7|
| 92616|Dr. Strangelove o...|1964| 8.7|
|250612|      Paths of Glory|1957| 8.6|
| 65764| Clockwork Orange, A|1971| 8.3|
| 30431|        Barry Lyndon|1975| 7.9|
|177019|        Killing, The|1956| 8.1|
|193519|              Lolita|1962| 7.6|
|176711|   Kill Bill: Vol. 1|2003| 8.4|
|176712|   Kill Bill: Vol. 2|2004| 8.2|
|267038|        Pulp Fiction|1994| 8.7|
|276217|      Reservoir Dogs|1992| 8.3|
+------+--------------------+----+----+
only showing top 20 rows



# Renaming Attributes

In [28]:
exp12 = Renaming_Attributes_Spark(exp11, ["movies<-name","published<-year"])

In [29]:
exp12.evaluate().show()

+--------------------+---------+
|              movies|published|
+--------------------+---------+
|           Spartacus|     1960|
|   Full Metal Jacket|     1987|
|              Aliens|     1986|
|        Jackie Brown|     1997|
|2001: A Space Ody...|     1968|
|Expedition: Bismarck|     2002|
|     Terminator, The|     1984|
| Inglorious Bastards|     2006|
|Terminator 2: Jud...|     1991|
|                  ER|     1994|
|Dr. Strangelove o...|     1964|
|      Paths of Glory|     1957|
| Clockwork Orange, A|     1971|
|        Barry Lyndon|     1975|
|        Killing, The|     1956|
|              Lolita|     1962|
|   Kill Bill: Vol. 1|     2003|
|   Kill Bill: Vol. 2|     2004|
|        Pulp Fiction|     1994|
|      Reservoir Dogs|     1992|
+--------------------+---------+
only showing top 20 rows



# Theta Join

In [30]:
directorsAndTheirMovies = Theta_Join_Spark(directors, movies_directors, "id==director_id")

In [31]:
directorsAndTheirMovies.evaluate().show(10)

+-----+----------+---------+-----------+--------+
|   id|first_name|last_name|director_id|movie_id|
+-----+----------+---------+-----------+--------+
|43095|   Stanley|  Kubrick|      43095|   30431|
|43095|   Stanley|  Kubrick|      43095|   92616|
|43095|   Stanley|  Kubrick|      43095|    1711|
|43095|   Stanley|  Kubrick|      43095|  176891|
|43095|   Stanley|  Kubrick|      43095|  110246|
|43095|   Stanley|  Kubrick|      43095|  177019|
|43095|   Stanley|  Kubrick|      43095|   65764|
|43095|   Stanley|  Kubrick|      43095|  106666|
|43095|   Stanley|  Kubrick|      43095|  121538|
|43095|   Stanley|  Kubrick|      43095|  310455|
+-----+----------+---------+-----------+--------+
only showing top 10 rows



# Equi Join

In [32]:
directorsAndTheirMovies = Equi_Join_Spark(directors, movies_directors, ['id'], ['director_id'])

In [33]:
directorsAndTheirMovies.evaluate().show(10)

+-----+----------+---------+-----------+--------+
|   id|first_name|last_name|director_id|movie_id|
+-----+----------+---------+-----------+--------+
|43095|   Stanley|  Kubrick|      43095|   30431|
|43095|   Stanley|  Kubrick|      43095|   92616|
|43095|   Stanley|  Kubrick|      43095|    1711|
|43095|   Stanley|  Kubrick|      43095|  176891|
|43095|   Stanley|  Kubrick|      43095|  110246|
|43095|   Stanley|  Kubrick|      43095|  177019|
|43095|   Stanley|  Kubrick|      43095|   65764|
|43095|   Stanley|  Kubrick|      43095|  106666|
|43095|   Stanley|  Kubrick|      43095|  121538|
|43095|   Stanley|  Kubrick|      43095|  310455|
+-----+----------+---------+-----------+--------+
only showing top 10 rows



# Grouping

In [34]:
# Idea: count the number of female/male actors

# grouping key: ['gender']
# aggregation function: len (also called count)
# notice that for len() specifying an attribute is actually not required
# as only the number of tuples in each group are coiunted
# this is independent of a specific attribute value
grouping = Grouping_Spark(actors, ['gender'], [(len, 'id')])

In [35]:
grouping.evaluate().show()

+------+---------+
|gender|count(id)|
+------+---------+
|     F|      289|
|     M|      802|
+------+---------+



In [36]:
# Idea: count the number of movies per year
# also sum up the years in each group which
# does not make sense semantically, just done to
# show that one can use multiple aggregations in a Grouping

# grouping key: ['year']
# aggregation function: len (also called count)
# aggregation function: sum (also called count)
grouping = Grouping_Spark(movies, ['year'], [(len, 'year'), (sum, 'id')])

In [37]:
grouping.evaluate().show()

+----+-----------+-------+
|year|count(year)|sum(id)|
+----+-----------+-------+
|1975|          1|  30431|
|2003|          3| 697546|
|1955|          1| 176891|
|2006|          1| 159665|
|1978|          1| 369522|
|1956|          1| 177019|
|1997|          2| 498428|
|1994|          3| 995565|
|1968|          1|   1711|
|1951|          2| 196085|
|1971|          1|  65764|
|2004|          1| 176712|
|1991|          1| 328277|
|1957|          1| 250612|
|1989|          1|   5306|
|1996|          1| 322652|
|1960|          1| 310455|
|1987|          2| 345248|
|1995|          1| 118367|
|1980|          1| 299073|
+----+-----------+-------+
only showing top 20 rows



## Query Optimization in Spark

Spark SQL implements its own query optimizer. It performs the following steps:
1. Parse the query and generate the logical plan.
2. Analyze the logical plan. In this step, the schema is checked and types are resolved.
3. Optimize the logical plan using rule-based optimization, e.g. by applying selection push-down
4. Generate multiple possible physical plans and pick the cheapest one with respect to a cost-model. 

In [38]:
# spark plan 
cp = Cartesian_Product_Spark(directors, directors_genres)
sel1 = Selection_Spark(cp, "id == director_id")
sel2 = Selection_Spark(sel1, "last_name == 'Tarantino' and genre == 'Mystery'")
proj = Projection_Spark(sel2, ['last_name', 'prob'])

In [39]:
# print logical and physical plan
proj.evaluate().explain(True)

== Parsed Logical Plan ==
'Project [unresolvedalias('last_name, None), unresolvedalias('prob, None)]
+- Filter ((last_name#14 = Tarantino) && (genre#23 = Mystery))
   +- Filter (id#12 = director_id#22)
      +- Join Cross
         :- LogicalRDD [id#12, first_name#13, last_name#14], false
         +- LogicalRDD [director_id#22, genre#23, prob#24], false

== Analyzed Logical Plan ==
last_name: string, prob: float
Project [last_name#14, prob#24]
+- Filter ((last_name#14 = Tarantino) && (genre#23 = Mystery))
   +- Filter (id#12 = director_id#22)
      +- Join Cross
         :- LogicalRDD [id#12, first_name#13, last_name#14], false
         +- LogicalRDD [director_id#22, genre#23, prob#24], false

== Optimized Logical Plan ==
Project [last_name#14, prob#24]
+- Join Cross, (id#12 = director_id#22)
   :- Project [id#12, last_name#14]
   :  +- Filter (last_name#14 = Tarantino)
   :     +- LogicalRDD [id#12, first_name#13, last_name#14], false
   +- Project [director_id#22, prob#24]
      +- Fi

## Logical to Spark Plan Compilation

In [40]:
# one to one translation from our logical operators to our spark operators.
def logical_to_spark(op):
    if(isinstance(op, LeafOperator)): return LeafSpark(op.relation, session)
    elif(isinstance(op, Selection)): return Selection_Spark(op.input, op.predicate)
    elif(isinstance(op, Projection)): return Projection_Spark(op.input, op.attributes)
    elif(isinstance(op, Cartesian_Product)): return Cartesian_Product_Spark(op.l_input, op.r_input) 
    elif(isinstance(op, SetOperator)): return SetOperator_Spark(op.l_input, op.r_input, op.operator, op.symbol)
    elif(isinstance(op, Renaming_Relation)): return Renaming_Relation_Spark(op.input, op.name)
    elif(isinstance(op, Renaming_Attributes)): return Renaming_Attributes_Spark(op.input, op.changes) 
    elif(isinstance(op, Theta_Join)): return Theta_Join_Spark(op.l_input, op.r_input, op.theta) 
    elif(isinstance(op, Equi_Join)): return Equi_Join_Spark(op.l_input, op.r_input, op.l_attrs, op.r_attrs)
    elif(isinstance(op, Grouping)): return Grouping_Spark(op.input, op.group_by, op.aggregations)    
    else: return None

# compile a logical operator tree to a spark operator tree
def compile_to_spark(op):
    new_op = logical_to_spark(op)
    if(isinstance(op, UnaryOperator)):
        new_op.input = compile_to_spark(op.input)
    elif(isinstance(op, BinaryOperator)):
        new_op.l_input = compile_to_spark(op.l_input)
        new_op.r_input = compile_to_spark(op.r_input)
    return new_op

# logical plan
cp = Cartesian_Product(directors, directors_genres)
sel1 = Selection(cp, "id == director_id")
sel2 = Selection(sel1, "last_name == 'Tarantino' and genre == 'Mystery'")
proj = Projection(sel2, ['last_name', 'prob'])

# compile
phys_root = compile_to_spark(proj)

# run 
phys_root.evaluate().show()

+---------+-----+
|last_name| prob|
+---------+-----+
|Tarantino|0.125|
+---------+-----+



In [41]:
phys_root.evaluate().explain(True)

== Parsed Logical Plan ==
'Project [unresolvedalias('last_name, None), unresolvedalias('prob, None)]
+- Filter ((last_name#374 = Tarantino) && (genre#379 = Mystery))
   +- Filter (id#372 = director_id#378)
      +- Join Cross
         :- LogicalRDD [id#372, first_name#373, last_name#374], false
         +- LogicalRDD [director_id#378, genre#379, prob#380], false

== Analyzed Logical Plan ==
last_name: string, prob: float
Project [last_name#374, prob#380]
+- Filter ((last_name#374 = Tarantino) && (genre#379 = Mystery))
   +- Filter (id#372 = director_id#378)
      +- Join Cross
         :- LogicalRDD [id#372, first_name#373, last_name#374], false
         +- LogicalRDD [director_id#378, genre#379, prob#380], false

== Optimized Logical Plan ==
Project [last_name#374, prob#380]
+- Join Cross, (id#372 = director_id#378)
   :- Project [id#372, last_name#374]
   :  +- Filter (last_name#374 = Tarantino)
   :     +- LogicalRDD [id#372, first_name#373, last_name#374], false
   +- Project [dire

## SQL in Spark

It is also possible to pass entire SQL queries to Spark. To use a DataFrame in a query, it must be named with createOrReplaceTempView(). 

In [42]:
directors.df.createOrReplaceTempView("directors")
directors_genres.df.createOrReplaceTempView("directors_genres")

res = session.sql("SELECT last_name, prob \
                   FROM directors, directors_genres \
                   WHERE id == director_id and last_name == 'Tarantino' and genre == 'Mystery'")
res.show()

+---------+-----+
|last_name| prob|
+---------+-----+
|Tarantino|0.125|
+---------+-----+



## Performance in Spark

### SparkUI

Under <a href="http://localhost:4040/">http://localhost:4040/</a> the SparkUI shows execution details of the job. 

1. For the previous query, a job is created. 
2. This job is split into three stages.
   The first stage builds the "directors" data source and applies the selection last_name == 'Tarantino'.
   In parallel, the second stage builds the "directors_genres" data source and applies the selection genre == 'Mystery'.
   The third stage pulls the results of the two previous stages and performs the join.
3. The operators within a stage are processed in parallel on horizontally partitioned data.

**The following material was not covered/shown in the lecture:**

### Query Execution Time: Naive vs Parquet

In Spark, the execution time of a query largely depends on the underlying data source. So far, the Spark DataFrames were internally wrapping Spark RDDs, which are the most fundamental data source in Spark. They simply return the entire dataset row by row. 

Let us see what this means for the performance. The following query joins three tables and performs two selections. We measure the runtime using %%time.

In [43]:
actors.df.createOrReplaceTempView("actors")
movies.df.createOrReplaceTempView("movies")
roles.df.createOrReplaceTempView("roles")

In [44]:
%%time
# query
res = session.sql("SELECT first_name, last_name, role \
                   FROM actors, roles, movies \
                   WHERE actors.id == roles.actor_id \
                     AND roles.movie_id == movies.id \
                     AND gender == 'M' \
                     AND year >= 1995")
res.show()

+-----------------+---------+--------------------+
|       first_name|last_name|                role|
+-----------------+---------+--------------------+
|       Jorge (II)|    Silva|      Bartender/Pimp|
|            Clark|Middleton|               Ernie|
|      Michael (I)|    Parks|      Esteban Vihaio|
|        Samuel L.|  Jackson|               Rufus|
|              Sid|     Haig|                 Jay|
|               Bo|  Svenson|    Reverend Harmony|
|        Al Manuel|  Douglas|     Marty Kitrosser|
|         Chia Hui|      Liu|             Pai Mei|
|            Stevo|    Polyi|                 Tim|
|            Larry|   Bishop|         Larry Gomez|
|      Michael (I)|   Madsen|   Budd (Sidewinder)|
|            David|Carradine|Bill AKA Snake Ch...|
|Christopher Allen|   Nelson|      Tommy Plympton|
|     William Paul|    Clark|           Soda Jerk|
|        James (I)|  Cameron|             Himself|
|      Justin (II)|    Baker|        Harold Bride|
|             Eric|  Schmitz|  

The following explain shows, that on the leaf level, a "Scan ExistingRDD" is performed, which simply returns all rows. 

In [45]:
res.explain()

== Physical Plan ==
*(8) Project [first_name#5, last_name#6, role#38]
+- *(8) SortMergeJoin [movie_id#37], [id#28], Inner
   :- *(5) Sort [movie_id#37 ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(movie_id#37, 200)
   :     +- *(4) Project [first_name#5, last_name#6, movie_id#37, role#38]
   :        +- *(4) SortMergeJoin [id#4], [actor_id#36], Inner
   :           :- *(2) Sort [id#4 ASC NULLS FIRST], false, 0
   :           :  +- Exchange hashpartitioning(id#4, 200)
   :           :     +- *(1) Project [id#4, first_name#5, last_name#6]
   :           :        +- *(1) Filter (gender#7 = M)
   :           :           +- Scan ExistingRDD[id#4,first_name#5,last_name#6,gender#7]
   :           +- *(3) Sort [actor_id#36 ASC NULLS FIRST], false, 0
   :              +- Exchange hashpartitioning(actor_id#36, 200)
   :                 +- Scan ExistingRDD[actor_id#36,movie_id#37,role#38]
   +- *(7) Sort [id#28 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(id#28, 2

To generate a more efficient physical plan, the Parquet (<a href="https://parquet.apache.org/">https://parquet.apache.org/</a>) data source can be used. Due to its internal organization, Parquet enables the following optimizations:
1. Column Projection: Only the columns, that are actually required by the query, are read and returned to the operator above.
2. Partition Pruning: The data can be partitioned horizontally. As a consequence, only those partitions, which are actually required by the query, are read and returned to the operator above. 
3. Selection Pushdown: Parquet builds statistics on its data, such as min and max value per chunk of data. These statistics can be used to prune chunks of data, which can not contain qualifying entries. 

In [46]:
# Write Parquet files (which are actually directories) from our existing DataFrames

# Partition actors by gender. This rules out half the partitions for queries selecting on gender
actors.df.write.mode("overwrite").partitionBy("gender").parquet("data/parquet/actors.parquet")

# No partitioning
movies.df.write.mode("overwrite").parquet("data/parquet/movies.parquet")

# No partitioning
roles.df.write.mode("overwrite").parquet("data/parquet/roles.parquet")

In [47]:
# read the Parquet files again and bind them to DataFrames
actorsParquetDF = session.read.parquet("data/parquet/actors.parquet")
moviesParquetDF = session.read.parquet("data/parquet/movies.parquet")
rolesParquetDF = session.read.parquet("data/parquet/roles.parquet")

In [48]:
# create temporary views to access them via spark SQL
actorsParquetDF.createOrReplaceTempView("actors_parquet")
moviesParquetDF.createOrReplaceTempView("movies_parquet")
rolesParquetDF.createOrReplaceTempView("roles_parquet")

In [49]:
%%time

# query
res = session.sql("SELECT first_name, last_name, role \
                   FROM actors_parquet, roles_parquet, movies_parquet \
                   WHERE actors_parquet.id == roles_parquet.actor_id \
                     AND roles_parquet.movie_id == movies_parquet.id \
                     AND gender == 'M' \
                     AND year >= 1995")
res.show()

+-----------+--------------+--------------------+
| first_name|     last_name|                role|
+-----------+--------------+--------------------+
|      Heinz|         Steeg|             Himself|
|   Seth (I)|        Adkins|Slovakian three-y...|
|Michael (I)|        Madsen|      Babe Buchinsky|
|    Xiaohui|            Hu|Young 88 (Spanked...|
|  James (I)|       Cameron|             Himself|
|  James (I)|       Cameron|             Himself|
|       Marc|          Cass|     Hold steward #1|
|   Leonardo|      DiCaprio|         Jack Dawson|
|     Victor|        Garber|      Thomas Andrews|
|     Arnold|Schwarzenegger|      The Terminator|
|  Chris (I)|         Byrne|   Stairwell steward|
|      Blake|        Sutton| Young Walter Weintz|
|    Michael|         Biehn|          Kyle Reese|
|      Clark|     Middleton|               Ernie|
|   Aaron C.|    Fitzgerald|          Fred Fleet|
|       Rudy|       Joffroy|      'Safe' Bellboy|
|   John (I)|         Bruno|             Himself|


The query should execute faster now, even on the small files we are using. 

Let us now inspect the changes in the plan. These are:
1. The leaf is now called "FileScan parquet".
2. FileScan parquet reads and returns only the columns accessed by the query, e.g. id and year of movies.
3. FileScan parquet of actors reads and returns only one partition, namely the one containing all female actors.
4. Filescan parquet of movies uses statistics to evaluate GreaterThanOrEqual(year,1995), such that only qualifying entries are returned.

In [50]:
res.explain()

== Physical Plan ==
*(3) Project [first_name#441, last_name#442, role#458]
+- *(3) BroadcastHashJoin [movie_id#457], [id#448], Inner, BuildRight
   :- *(3) Project [first_name#441, last_name#442, movie_id#457, role#458]
   :  +- *(3) BroadcastHashJoin [id#440], [actor_id#456], Inner, BuildLeft
   :     :- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, true] as bigint)))
   :     :  +- *(1) Project [id#440, first_name#441, last_name#442]
   :     :     +- *(1) Filter isnotnull(id#440)
   :     :        +- *(1) FileScan parquet [id#440,first_name#441,last_name#442,gender#443] Batched: true, Format: Parquet, Location: InMemoryFileIndex[file:/Users/jens/Desktop/bigdataengineering.git/trunk/data/parquet/actors.parquet], PartitionCount: 1, PartitionFilters: [isnotnull(gender#443), (gender#443 = M)], PushedFilters: [IsNotNull(id)], ReadSchema: struct<id:int,first_name:string,last_name:string>
   :     +- *(3) Project [actor_id#456, movie_id#457, role#458]
   :        +-